# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Nov 14 2022 10:29AM,250667,10,0086166986,ISDIN Corporation,Released
1,Nov 14 2022 10:29AM,250667,10,0086167038,ISDIN Corporation,Released
2,Nov 14 2022 10:29AM,250667,10,0086167037,ISDIN Corporation,Released
3,Nov 14 2022 10:29AM,250667,10,0086167057,ISDIN Corporation,Released
4,Nov 14 2022 10:29AM,250667,10,0086167058,ISDIN Corporation,Released
5,Nov 14 2022 10:29AM,250667,10,0086167061,ISDIN Corporation,Released
6,Nov 14 2022 10:29AM,250667,10,0086167051,ISDIN Corporation,Released
7,Nov 14 2022 10:29AM,250667,10,0086167052,ISDIN Corporation,Released
8,Nov 14 2022 10:29AM,250667,10,0086167053,ISDIN Corporation,Released
9,Nov 14 2022 10:29AM,250667,10,0086167054,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
18,250669,Released,4
19,250671,Released,6
20,250672,Released,3
21,250673,Released,1
22,250674,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
250669,NaN,4.0
250671,NaN,6.0
250672,NaN,3.0
250673,NaN,1.0
250674,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
250634,0.0,2.0
250638,0.0,5.0
250648,0.0,1.0
250649,1.0,0.0
250651,47.0,17.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
250634,0,2
250638,0,5
250648,0,1
250649,1,0
250651,47,17


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,250634,0,2
1,250638,0,5
2,250648,0,1
3,250649,1,0
4,250651,47,17


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,250634,,2
1,250638,,5
2,250648,,1
3,250649,1,
4,250651,47,17


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Nov 14 2022 10:29AM,250667,10,ISDIN Corporation
54,Nov 14 2022 10:29AM,250666,10,ISDIN Corporation
122,Nov 14 2022 10:29AM,250662,10,ISDIN Corporation
180,Nov 14 2022 10:25AM,250674,21,"NBTY Global, Inc."
181,Nov 14 2022 10:25AM,250673,21,"NBTY Global, Inc."
182,Nov 14 2022 10:23AM,250672,15,"Mizner Bioscience, LLC"
185,Nov 14 2022 10:22AM,250671,15,"Brookfield Pharmaceuticals, LLC"
191,Nov 14 2022 10:21AM,250653,16,Sartorius Bioprocess Solutions
192,Nov 14 2022 10:16AM,250668,10,Bio-PRF
229,Nov 14 2022 10:08AM,250669,19,"Graystone, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Nov 14 2022 10:29AM,250667,10,ISDIN Corporation,,54
1,Nov 14 2022 10:29AM,250666,10,ISDIN Corporation,,68
2,Nov 14 2022 10:29AM,250662,10,ISDIN Corporation,,58
3,Nov 14 2022 10:25AM,250674,21,"NBTY Global, Inc.",,1
4,Nov 14 2022 10:25AM,250673,21,"NBTY Global, Inc.",,1
5,Nov 14 2022 10:23AM,250672,15,"Mizner Bioscience, LLC",,3
6,Nov 14 2022 10:22AM,250671,15,"Brookfield Pharmaceuticals, LLC",,6
7,Nov 14 2022 10:21AM,250653,16,Sartorius Bioprocess Solutions,,1
8,Nov 14 2022 10:16AM,250668,10,Bio-PRF,,37
9,Nov 14 2022 10:08AM,250669,19,"Graystone, LLC",,4


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Nov 14 2022 10:29AM,250667,10,ISDIN Corporation,54,
1,Nov 14 2022 10:29AM,250666,10,ISDIN Corporation,68,
2,Nov 14 2022 10:29AM,250662,10,ISDIN Corporation,58,
3,Nov 14 2022 10:25AM,250674,21,"NBTY Global, Inc.",1,
4,Nov 14 2022 10:25AM,250673,21,"NBTY Global, Inc.",1,
5,Nov 14 2022 10:23AM,250672,15,"Mizner Bioscience, LLC",3,
6,Nov 14 2022 10:22AM,250671,15,"Brookfield Pharmaceuticals, LLC",6,
7,Nov 14 2022 10:21AM,250653,16,Sartorius Bioprocess Solutions,1,
8,Nov 14 2022 10:16AM,250668,10,Bio-PRF,37,
9,Nov 14 2022 10:08AM,250669,19,"Graystone, LLC",4,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Nov 14 2022 10:29AM,250667,10,ISDIN Corporation,54,
1,Nov 14 2022 10:29AM,250666,10,ISDIN Corporation,68,
2,Nov 14 2022 10:29AM,250662,10,ISDIN Corporation,58,
3,Nov 14 2022 10:25AM,250674,21,"NBTY Global, Inc.",1,
4,Nov 14 2022 10:25AM,250673,21,"NBTY Global, Inc.",1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Nov 14 2022 10:29AM,250667,10,ISDIN Corporation,54.0,NaN
1,Nov 14 2022 10:29AM,250666,10,ISDIN Corporation,68.0,NaN
2,Nov 14 2022 10:29AM,250662,10,ISDIN Corporation,58.0,NaN
3,Nov 14 2022 10:25AM,250674,21,"NBTY Global, Inc.",1.0,NaN
4,Nov 14 2022 10:25AM,250673,21,"NBTY Global, Inc.",1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Nov 14 2022 10:29AM,250667,10,ISDIN Corporation,54.0,0.0
1,Nov 14 2022 10:29AM,250666,10,ISDIN Corporation,68.0,0.0
2,Nov 14 2022 10:29AM,250662,10,ISDIN Corporation,58.0,0.0
3,Nov 14 2022 10:25AM,250674,21,"NBTY Global, Inc.",1.0,0.0
4,Nov 14 2022 10:25AM,250673,21,"NBTY Global, Inc.",1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,2757224,421.0,47.0
15,752007,27.0,0.0
16,250653,1.0,0.0
19,1002618,6.0,1.0
20,250663,1.0,0.0
21,501347,2.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,2757224,421.0,47.0
1,15,752007,27.0,0.0
2,16,250653,1.0,0.0
3,19,1002618,6.0,1.0
4,20,250663,1.0,0.0
5,21,501347,2.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,421.0,47.0
1,15,27.0,0.0
2,16,1.0,0.0
3,19,6.0,1.0
4,20,1.0,0.0
5,21,2.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,421.0
1,15,Released,27.0
2,16,Released,1.0
3,19,Released,6.0
4,20,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,20,21
Status,,,,,,
Executing,47.0,0.0,0.0,1.0,0.0,0.0
Released,421.0,27.0,1.0,6.0,1.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,20,21
0,Executing,47.0,0.0,0.0,1.0,0.0,0.0
1,Released,421.0,27.0,1.0,6.0,1.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,20,21
0,Executing,47.0,0.0,0.0,1.0,0.0,0.0
1,Released,421.0,27.0,1.0,6.0,1.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()